In [ ]:
import numpy as np
import time
import pandas as pd
from datetime import datetime

In [ ]:
from bs4 import BeautifulSoup
from bs4 import element
import urllib
import re

html_page = urllib.request.urlopen("http://etcsl.orinst.ox.ac.uk/cgi-bin/etcsl.cgi?text=all#")
soup = BeautifulSoup(html_page)

In [ ]:
link_list = []

for link in soup.findAll('a'):
    link_list.append(link.get('href'))

In [ ]:
pref = "http://etcsl.orinst.ox.ac.uk/cgi-bin/"
paired_links = []

for i in range(len(link_list)-1):
    if "text=c" in link_list[i] and "text=t" in link_list[i+1]:
        paired_links.append(pref + link_list[i])
        paired_links.append(pref + link_list[i+1])

In [ ]:
link_pairs = np.reshape(np.array(paired_links), (-1,2))

In [ ]:
# first parse translation to understand what sentences should be concatenated.

tips = []

# http://etcsl.orinst.ox.ac.uk/cgi-bin/etcsl.cgi?text=c.1.1.2&display=Crit&charenc=gcirc#

for i in range(len(link_pairs)):  
    
    print(i, "\t", link_pairs[i][0])
    html_page = urllib.request.urlopen(link_pairs[i][0])
    soup = BeautifulSoup(html_page, "html5") # "xml" parser is significantly faster but freezes.
    
    for span in soup.findAll('span'):
        tip = span.get('onMouseover')
        if not tip == None:
            tip = tip[18:-2]
            tip = re.sub("<sub>|</sub>", "", tip)
            tip = re.sub(" \(.*\) ", ";", tip)
            tip = re.sub(";to be ", ";", tip)
            tip = re.sub(";to ", ";", tip)
            tip = re.sub("soothing expression", "oh", tip)
            tip = re.sub("\?", "", tip)

            ## information loss
            tip = re.sub(" *\(.+\)", "", tip)
            
            tips.append(tip)
    
    if i%3 == 2:
        print("i sleep")
        time.sleep(10)

print("{}\ttokens".format(len(tips)))
unique_tips = np.unique(tips)
print("{}\tunique tokens".format(len(unique_tips)))

## Inspect this closer to remove unwanted pairs.

unique_pairs = [i.replace('', '').split(";") for i in unique_tips]

# If RHS has "," then split RHS and add both parts as different pairs
for pair in unique_pairs:
    if len(pair)!=2:
        unique_pairs.remove(pair)
    elif "," in pair[1]:
        rhs = pair[1].split(", ")
        unique_pairs.append([pair[0], rhs[0]])
        unique_pairs.append([pair[0], rhs[1]])
        unique_pairs.remove(pair)
        
now   = datetime.now() # current date and time
stamp = now.strftime("%Y%m%d%H%M")        

pd.DataFrame(unique_pairs).to_csv("data/unique_tip_pairs_all.csv", ";", index=None, header=["akk", "en"])

In [ ]:
token_pairs = pd.read_csv("data/unique_tip_pairs_all.csv", ";")

In [ ]:
# Returns True if given "num", line number is between a given line number range.
# Takes into consideration the line numbers that end in letters, i.e. "49A-49J."

def lesstoe(ran, num):
    
    if len(re.findall(r"[0-9]*-?[0-9]*\.", ran)) == 0 or len(re.findall(r"[0-9]*\.", num)) == 0:
        return True
    
        
    offset = 54
    
    ran = ran[:-1]
    
    if num == "":
        num = "p"
    
    if not num[0].isdigit():
        num = "0."
        
    num = num[:-1]
    
    drange = ran.split("-")
    
    
    if len(drange) == 2:
        drange.append(num)
        

        for i in range(3):
            
            if len(drange[i])>2:
                if drange[i][-2].isalpha():
                    drange[i] = drange[i][:-2] + "." + repr(9) + drange[i][-1]
                    drange[i] = drange[i][:4] + re.sub("\.", "", drange[i][4:])
            
            if len(drange[i])>1:
                if drange[i][-1].isalpha():
                    drange[i] = re.sub(drange[i][-1], "." + repr(ord(drange[i][-1])-offset),drange[i])
                    drange[i] = drange[i][:4] + re.sub("\.", "", drange[i][4:])

                    
        #print(drange)
        try:
            return (float(drange[2]) <= float(drange[1])) or float(drange[2]) == 0
        except:
            print(drange[2], drange[1], drange[2])

    else:
        #print(drange)
        return ran != num

In [ ]:
conts_list     = []
sentences_list = []
ltrs_list      = []
j=0

In [ ]:
j=348
conts_list = pd.read_csv("data/sentence_pairs_{}.csv".format(j), ";").to_numpy().tolist()

In [ ]:


#j=0 works 
while j!=len(link_pairs):
    
    #translation

    conts = []
    nums = []

    print(j, "\t", link_pairs[j][1])
    html_page = urllib.request.urlopen(link_pairs[j][1])
    soup = BeautifulSoup(html_page, "html5") # "xml" parser is significantly faster but freezes.

    for a in soup.findAll("p"):
        c = a.contents
        if len(c) != 0:
            for i in range(len(c)):
                if type(c[i]) is element.Tag:
                    try:
                        c[i] = "".join(c[i].contents)
                    except:
                        for k in range(len(c[i])):
                            s = str(c[i].contents[k])
                            s = re.sub("<span class=\"note\">.*?\"proper\">|</span></span>", "", s)
                            c[i].contents[k] = s
                        c[i] = "".join(c[i].contents)
            conts.append(c)

    for a in soup.findAll("a"):
        c = a.contents
        if len(c) != 0:
            if c[0].isdigit:
                nums.append(c)

    for i in range(len(conts)):
        conts[i][1] = "".join(conts[i][1:])
        conts[i][1] = re.sub("\(.*?\)", "", conts[i][1])
        conts[i][1] = re.sub("  ", " ", conts[i][1])
        del conts[i][3:]

    del conts[-1]



    ## transliteration

    ltrs = []

    print(j, "\t", link_pairs[j][0])
    html_page = urllib.request.urlopen(link_pairs[j][0])
    soup = BeautifulSoup(html_page, "html5") # "xml" parser is significantly faster but freezes.

    for a in soup.findAll("td"):
        c = a.contents
        if len(c) != 0:
            for i in range(len(c)):
                if type(c[i]) is element.Tag:
                    s = str(c[i])

                    s = re.sub("<span onMouseover.*?;", "", s)

                    s = re.sub(" -- ", " ", s)
                    s = re.sub("<phr.*/phr>", "", s)
                    s = re.sub("<font.*/font>", "", s)
                    s = re.sub("<a.*?>|</a>", "", s)
                    s = re.sub("<img.*?>", "", s)
                    s = re.sub("\n", "", s)
                    s = re.sub("<span.*?>|</span>", "", s)
                    s = re.sub("<sup>", "^", s)
                    s = re.sub("</sup>", "-", s)
                    s = re.sub("<sub>|</sub>", "", s)

                    c[i] = s
        ltrs.append(c)

    for i in range(len(ltrs)):
        ltrs[i] = "".join(ltrs[i])


    ## concatenated transliterations



    k = 0

    l = len(ltrs)

    sentences = []

    for ran in nums:

        ran =  ran[0]

        sentence = ""

        while(k<l-1 and lesstoe(ran, ltrs[k])):
            if len(ltrs[k])>1 and not bool(re.search('[0-9]*?[A-Z]*?\.', ltrs[k+1])):
                sentence += ltrs[k+1]
            k+=1

        sentences.append(sentence)


    for i in range(len(conts)):
        conts[i][2] = sentences[i]

    ltrs_list.extend(ltrs)
    conts_list.extend(conts)
    sentences_list.extend(sentences)
    
    
    if j%4 == 0:
        pd.DataFrame(conts_list).to_csv("data/sentence_pairs_{}.csv".format(j), ";", index=None, header=["index", "en", "akk"])
    #print("i sleep")
    #time.sleep(10)
    j+=1

In [ ]:
#problem, last line not going through in some cases.
conts_list

In [ ]:
# try with tokenizing using tirets & not, compare the results.

In [ ]:
conts_list_np = np.asarray(conts_list[:4436])

In [ ]:
import pandas as pd
df = pd.DataFrame(conts_list_np[:,1:], columns=["en", "akk"])
df = df.append(token_pairs, sort=True, ignore_index=True)
df.replace('', np.nan, inplace=True)
df.dropna(inplace=True)

In [ ]:
df.to_csv("data/6882pairs.csv", ";", index=False)